In [70]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

import matplotlib.pyplot as plt

import numpy as np
import cPickle as pickle
import sys
import os
import yaml

sys.path.append('/Users/Michael/projects/shape_sharing/src/')
from common import paths
from common import voxel_data
from common import images
from common import parameters
from common import features

In [73]:
sequence = paths.RenderedData.train_sequence()[1]

# loading the information about this frame
frame_data = paths.RenderedData.load_scene_data(
    sequence['scene'], sequence['frames'][0])

# for now I will just use the first frame of frames as the one to use
im = images.RGBDImage.load_from_dict(
    paths.RenderedData.scenes_location + sequence['scene'], frame_data)
norm_engine = features.Normals()
im.normals = norm_engine.compute_normals(im)

In [74]:
print im.normals.shape
plt.imshow(im.normals[:, 0].reshape(im.depth.shape))
norms = im.normals.reshape((480, 640, 3))
plt.imshow(norms[:, :, 2])

In [128]:
hdiff_n = np.zeros(im.depth.shape)
hdiff_n_c = np.zeros(im.depth.shape)

xyz = im.reproject_3d().T.reshape((480, 640, 3))
#print xyz.shape

for i in range(480-1):
    temp = np.sum(norms[i, :, :] * norms[i+1, :, :], axis=1)
    temp[temp<0] = 0
    if i < 480 - 10:
        temp_dir = np.sum((xyz[i, :, :] - xyz[i+5, :, :]) * norms[i, :, :], axis=1)
    hdiff_n_c[i, :] = temp_dir > 0
    hdiff_n[i, temp_dir>=0] = (1 - temp[temp_dir>=0])**2
    hdiff_n[i, temp_dir<0] = (1 - temp[temp_dir<0])

vdiff_n = np.zeros(im.depth.shape)
vdiff_n_c = np.zeros(im.depth.shape)
for i in range(640-1):
    temp = np.sum(norms[:, i, :] * norms[:, i+1, :], axis=1)
    temp[temp<0] = 0
    if i < 640 - 10:
        temp_dir = np.sum((xyz[:, i, :] - xyz[:, i+5, :]) * norms[:, i, :], axis=1)
    vdiff_n_c[:, i] = temp_dir > 0
    vdiff_n[temp_dir>=0, i] = (1 - temp[temp_dir>=0])**2
    vdiff_n[temp_dir<0, i] = (1 - temp[temp_dir<0])

plt.rcParams['figure.figsize'] = (10, 10)
plt.imshow(~(hdiff_n**2 + vdiff_n**2 > 0.1) & (temp_mask))
plt.clim((0, 1))
plt.colorbar()
print hdiff_n_c

In [7]:
# but actually want to segment on the graph.. using skimage? sklearn?
import scipy.sparse as sps
dists = sps.lil_matrix((640*480, 640*480), dtype=np.float16)


In [12]:
H = im.get_world_xyz()[:, 2]
ground_plane = np.abs(H.reshape(im.mask.shape)) < 0.01

temp_mask = ~ground_plane & im.mask
plt.imshow(temp_mask)

In [66]:
hdiff = np.zeros(im.depth.shape)
for i in range(480-1):
    hdiff[i, :] = im.depth[i, :] - im.depth[i+1, :]

vdiff = np.zeros(im.depth.shape)
for i in range(640-1):
    vdiff[:, i] = im.depth[:, i] - im.depth[:, i+1]

ddiff = np.sqrt(hdiff**2 + vdiff**2)
plt.imshow(ddiff)

In [65]:
seg = ~((ddiff > 0.05) | ~temp_mask)
plt.imshow(seg)

from skimage.morphology import watershed
from skimage.measure import label
from skimage.feature import peak_local_max
from scipy import ndimage

distance = ndimage.distance_transform_edt(seg)
plt.imshow(distance)
local_maxi = peak_local_max(distance, indices=False, footprint=np.ones((60, 60)),labels=seg)
markers = label(local_maxi)
labels_ws = watershed(-distance, markers, mask=seg)

plt.imshow(labels_ws)